In [1]:
import requests
from pyquery import PyQuery as pq
from io import StringIO
import pandas as pd
import numpy as np
import math

In [2]:
def dailyExchangeReportCrawler(datestr):
    # input example : "20190102" or "20191122", and the crawler will crawl daily exhange data from TWSE&TPEX
    
    # TWSE
    r_TWSE = requests.post("https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=" + datestr + "&type=ALLBUT0999")
    df_TWSE = pd.read_csv(StringIO(r_TWSE.text.replace("=", "")), header = ["證券代號" in i for i in r_TWSE.text.split("\n")].index(True) - 1)
    # data clean : eliminate ETF & Special Stock
    df_TWSE = df_TWSE[[i == "1101" for i in df_TWSE["證券代號"]].index(True)::]
    df_TWSE.reset_index(drop = True, inplace = True)
    SpecStockIndex = []
    for i in df_TWSE["證券代號"]:
        if i.isdigit() == False:
            SpecStockIndex.append(df_TWSE["證券代號"].tolist().index(i))
    df_TWSE.drop(index = SpecStockIndex, axis = 0, inplace = True)
    df_TWSE.reset_index(drop = True, inplace = True)
    # Collect columns that we really need.
    df_TWSE["年月日"] = datestr[0:4] + "/" + datestr[4:6] + "/" + datestr[6:8]
    df_TWSE["成交量"] = [int(i.replace(",",""))//1000 for i in df_TWSE["成交股數"]]
    df_TWSE["漲跌(+/-)"] = [i.replace("X","")for i in df_TWSE["漲跌(+/-)"]]
    df_TWSE["漲跌價差"] = [str(i) for i in df_TWSE["漲跌價差"]]
    df_TWSE["股價漲跌"] = df_TWSE["漲跌(+/-)"] + df_TWSE["漲跌價差"]
    df_TWSE = df_TWSE[["證券代號", "證券名稱", "年月日", "開盤價", "最高價", "最低價", "收盤價", "成交量", "股價漲跌"]]
    
    # TPEX
    datestr_TPEX = str(int(datestr[0:4])-1911) + "/" + datestr[4:6] + "/" + datestr[6:8]
    r_TPEX = requests.get("http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_download.php?l=zh-tw&d=" + datestr_TPEX + "&s=0,asc,0")
    lines = r_TPEX.text.replace("\r", "").split("\n")
    df_TPEX = pd.read_csv(StringIO("\n".join(lines[3:])), header=None)
    df_TPEX.columns = list(map(lambda i: i.replace(' ',''), lines[2].split(',')))
    # data clean : eliminate ETF & Special Stock
    # DR stock isn't included.
    cleanIndex = []
    for i in df_TPEX["代號"]:
        if (i.isdigit() == False) or (len(i) == 6) or (len(i) == 5):
            cleanIndex.append(df_TPEX["代號"].tolist().index(i))
    df_TPEX.drop(index = cleanIndex, axis = 0, inplace = True)
    df_TPEX.reset_index(drop = True, inplace = True)
    df_TPEX["證券代號"] = df_TPEX["代號"]
    df_TPEX["證券名稱"] = df_TPEX["名稱"]
    df_TPEX["年月日"] = datestr[0:4] + "/" + datestr[4:6] + "/" + datestr[6:8]
    df_TPEX["開盤價"] = df_TPEX["開盤"]
    df_TPEX["最高價"] = df_TPEX["最高"]
    df_TPEX["最低價"] = df_TPEX["最低"]
    df_TPEX["收盤價"] = df_TPEX["收盤"]
    df_TPEX["成交量"] = [int(i.replace(",",""))//1000 for i in df_TPEX["成交股數"]]
    df_TPEX["股價漲跌"] = df_TPEX["漲跌"]
    df_TPEX = df_TPEX[["證券代號", "證券名稱", "年月日", "開盤價", "最高價", "最低價", "收盤價", "成交量", "股價漲跌"]]
    
    # concat TWSE&TPEX, and reset the row index.
    df_All = pd.concat([df_TWSE, df_TPEX])
    df_All.reset_index(drop = True, inplace = True)
    return df_All
    # save file : df_ALL.to_csv("data.csv")
    """ Note:
    In this way, you can furthermore crawl periodical data.
    However, you need to notice that I haven't transformed Open/High/Low/Close data into float.
    That is because there might be "--- " in Open/High/Low/Close due to no transaction in day.
    So if you want to extend to craw periodical data, you need to be aware of this.
    You can transform "--- " base on yesterday's data, however, in the next step, you'll see another solution of it.
    """

In [3]:
def technicalAnalysisIndex(datacsv):
    # input example : "data.csv", which is your csv file name, and you have to aware that this csv needs to be seperated by "," and with encoding utf-8.
    # Also, the "data.csv" needs to be periodical data with more than 85 transaction days due to technical index convergence need.
    df = pd.read_csv(datacsv, encoding = "utf-8")
    SecurityList = sorted(set(df["證券代號"]))
    
    # 尋找成交量為0（停止交易或流動性差）的股票與發生日，並把index們存成一個名為zeroVol的list。
    zeroVol = df[df["成交量"] == 0].index.tolist()
    
    # 接著利用df.drop()來將原df中，0成交量的資料刪除，以便技術指標能正確被計算/判讀。
    df.drop(index = zeroVol, axis = 0, inplace = True)
    df.reset_index(drop = True, inplace = True)
    
    # calculate each stock's actual transaction days
    intervalDict = {}
    for key in df['證券代號']:
        intervalDict[key] = intervalDict.get(key,0) + 1
    
    # If a stock's actual transaction days is less than 85(due to new IPO), we will remove it, because we need to ensure the technical index's convergence.
    # So, the periodical data must be enough.
    NotSuff = []
    for i in intervalDict:
        if intervalDict[i] < 85:
            NotSuff.append(i)
            
    # remove NotSuff
    SecurityList = sorted(list(set(SecurityList).difference(set(NotSuff))))
    for i in NotSuff:
        intervalDict.pop(i)
    NotsuffIndex = []
    for i in NotSuff:
        NotsuffIndex += df[df["證券代號"] == i].index.tolist()
    df.drop(index = NotsuffIndex, axis = 0, inplace = True)
    df.reset_index(drop = True, inplace = True)
    
    # Start to calculate technical index, including KD, RSI, DIF-MACD, DMI, MA Offset, Bias
    #--------------------KD
    RSV = []
    for i in range(len(SecurityList)):
        for j in range(8):
            RSV.append("NA")
        for k in range(intervalDict[SecurityList[i]]-8):
            if i == 0:
                index = 8 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 8 + k
            if (max(df["最高價"][index-8:index+1])-min(df["最低價"][index-8:index+1])) == 0:
                RSVvalue = 50 #以防分母為0時。
            else:
                RSVvalue = (df["收盤價"][index]-min(df["最低價"][index-8:index+1]))/(max(df["最高價"][index-8:index+1])-min(df["最低價"][index-8:index+1]))*100
            RSV.append(RSVvalue)
    df["RSV9"] = RSV

    K = []
    for i in range(len(SecurityList)):
        for j in range(7):
            K.append("NA")
        K.append(50)#當作K值平滑起點
        for k in range(intervalDict[SecurityList[i]] - 8):
            if i == 0:
                index = 8 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 8 + k
            Kvalue = ((2/3)*K[-1]+(1/3)*df["RSV9"][index])
            K.append(Kvalue)
    df["K9"] = K

    D = []
    for i in range(len(SecurityList)):
        for j in range(7):
            D.append("NA")
        D.append(50)#當作D值平滑起點
        for k in range(intervalDict[SecurityList[i]] - 8):
            if i == 0:
                index = 8 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 8 + k
            Dvalue = ((2/3)*D[-1]+(1/3)*df["K9"][index])
            D.append(Dvalue)
    df["D9"] = D
    #------------------------------

    #--------------------6日&12日RSI
    Ut = [] #絕對漲幅
    Dt = [] #絕對跌幅
    for i in range(len(SecurityList)):
        Ut.append("NA")
        Dt.append("NA")
        for j in range(intervalDict[SecurityList[i]] - 1):
            if i == 0:
                index = 1 + j
            else:
                index = sum(list(intervalDict.values())[0:i]) + 1 + j
            Utvalue = float(max((df["收盤價"][index]-df["收盤價"][index-1]),0))
            Dtvalue = float(abs(min((df["收盤價"][index]-df["收盤價"][index-1]),0)))#因為要絕對值，不要負號。
            Ut.append(Utvalue)
            Dt.append(Dtvalue)
    df["Ut"] = Ut
    df["Dt"] = Dt

    UP_6D = [] #對絕對漲幅Ut利用“威爾德平滑法”做平滑平均，公式為：UPt = UPt-1 + (Ut - UPt-1)/N，N為RSI天數，通常拿6日跟12日
    DN_6D = [] #對絕對跌幅Dt利用“威爾德平滑法”做平滑平均，公式為：DNt = DNt-1 + (Dt - DNt-1)/N，N為RSI天數，通常拿6日跟12日
    for i in range(len(SecurityList)):
        for j in range(6):
            UP_6D.append("NA")
            DN_6D.append("NA") #第一日沒有Ut跟Dt，又接下來五日都無法計算6日平滑
        for k in range(intervalDict[SecurityList[i]] - 6):
            if i == 0:
                index = 6 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 6 + k
            if k == 0:
                UP_6Dvalue = sum(Ut[index-5:index+1])/6 #利用前6個Ut做簡單平均當作平滑的起點。
                DN_6Dvalue = sum(Dt[index-5:index+1])/6
                UP_6D.append(UP_6Dvalue)
                DN_6D.append(DN_6Dvalue)
            else:
                UP_6Dvalue = UP_6D[-1] + (df["Ut"][index] - UP_6D[-1])/6
                DN_6Dvalue = DN_6D[-1] + (df["Dt"][index] - DN_6D[-1])/6
                UP_6D.append(UP_6Dvalue)
                DN_6D.append(DN_6Dvalue)
    df["UP6"] = UP_6D
    df["DN6"] = DN_6D

    RSI6 = [] #計算6日RSI
    for i in range(len(SecurityList)):
        for j in range(6):
            RSI6.append("NA") #前6天算不出6日RSI
        for k in range(intervalDict[SecurityList[i]]-6):
            if i == 0:
                index = 6 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 6 + k
            RSI6value = (df['UP6'][index]/(df['UP6'][index]+df['DN6'][index]))*100
            RSI6.append(RSI6value)
    df["RSI6"] = RSI6

    UP_12D = [] 
    DN_12D = []
    for i in range(len(SecurityList)):
        for j in range(12):
            UP_12D.append("NA")
            DN_12D.append("NA") #第一日沒有Ut跟Dt，又接下來11日都無法計算12日平滑
        for k in range(intervalDict[SecurityList[i]] - 12):
            if i == 0:
                index = 12 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 12 + k
            if k == 0:
                UP_12Dvalue = sum(Ut[index-11:index+1])/12 #利用前12個Ut做簡單平均當作平滑的起點。
                DN_12Dvalue = sum(Dt[index-11:index+1])/12
                UP_12D.append(UP_12Dvalue)
                DN_12D.append(DN_12Dvalue)
            else:
                UP_12Dvalue = UP_12D[-1] + (Ut[index] - UP_12D[-1])/12
                DN_12Dvalue = DN_12D[-1] + (Dt[index] - DN_12D[-1])/12
                UP_12D.append(UP_12Dvalue)
                DN_12D.append(DN_12Dvalue)
    df["UP12"] = UP_12D
    df["DN12"] = DN_12D

    RSI12 = [] #計算12日RSI
    for i in range(len(SecurityList)):
        for j in range(12):
            RSI12.append("NA") #前12天算不出12日RSI
        for k in range(intervalDict[SecurityList[i]]-12):
            if i == 0:
                index = 12 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 12 + k
            RSI12value = (df['UP12'][index]/(df['UP12'][index]+df['DN12'][index]))*100
            RSI12.append(RSI12value)
    df["RSI12"] = RSI12
    #------------------------------

    #--------------------DIF-MACD
    df["DI"] = (df["最高價"] + df['最低價'] + 2 * df['收盤價'])/4
    EMA12 = []
    EMA26 = []
    DIF = []
    MACD = []
    OSC = []
    for i in range(len(SecurityList)):
        for j in range(11):
            EMA12.append("NA") #前11日不會有EMA
        for k in range(25):
            EMA26.append("NA") #前25日不會有EMA&DIF
            DIF.append("NA")
        for l in range(intervalDict[SecurityList[i]]-11):
            if i == 0:
                index = 11 + l
            else:
                index = sum(list(intervalDict.values())[0:i]) + 11 + l
            if l == 0:
                EMA12value = sum(df["DI"][index-11:index+1])/12 #首日做加權平均
                EMA12.append(EMA12value)
            else:
                EMA12value = (EMA12[-1] * 11 + df["DI"][index] * 2)/13
                EMA12.append(EMA12value)
        for m in range(intervalDict[SecurityList[i]]-25):
            if i == 0:
                index = 25 + m
            else:
                index = sum(list(intervalDict.values())[0:i]) + 25 + m
            if m == 0:
                EMA26value = sum(df["DI"][index-25:index+1])/26
                EMA26.append(EMA26value)
            else:
                EMA26value = (EMA26[-1] * 25 + df["DI"][index] * 2)/27
                EMA26.append(EMA26value)
        for n in range(intervalDict[SecurityList[i]]-25):
            if i == 0:
                index = 25 + n
            else:
                index = sum(list(intervalDict.values())[0:i]) + 25 + n
            DIFvalue = EMA12[index] - EMA26[index]
            DIF.append(DIFvalue)
        for o in range(25+8):
            MACD.append("NA") #前25日都沒有DIF，加上是9日MACD，故在有DIF的前8日也無法算9日MACD。
            OSC.append("NA") #同時也都沒辦法算OSC
        for p in range(intervalDict[SecurityList[i]]-33):
            if i == 0:
                index = 33 + p
            else:
                index = sum(list(intervalDict.values())[0:i]) + 33 + p
            if p == 0:
                MACDvalue = sum(DIF[index-8:index+1])/9 #首日做加權平均
                MACD.append(MACDvalue)
            else:
                MACDvalue = (MACD[-1] * 8 + DIF[index] * 2)/10
                MACD.append(MACDvalue)
        for q in range(intervalDict[SecurityList[i]]-33):
            if i == 0:
                index = 33 + q
            else:
                index = sum(list(intervalDict.values())[0:i]) + 33 + q
            OSCvalue = DIF[index] -MACD[index]
            OSC.append(OSCvalue)
    df["EMA12"] = EMA12
    df["EMA26"] = EMA26
    df["DIF12-26"] = DIF
    df["MACD9"] = MACD
    df["OSC"] = OSC
    #------------------------------

    #--------------------均線扣抵&乖離率 （需plot圖）
    MA5 = []
    MA20 = []
    MA60 = []
    MA6 = []
    MA12 = []
    MA24 = []
    MA72 =[]
    #事實上，這些均線用不到，只是先做以防萬一。

    MA6_BIAS = []
    MA12_BIAS = []
    MA24_BIAS = []
    MA72_BIAS = []
    for i in range(len(SecurityList)):
        for j in range(4):
            MA5.append("NA")
        for k in range(19):
            MA20.append("NA")
        for l in range(59):
            MA60.append("NA")
        for m in range(intervalDict[SecurityList[i]]-4):
            if i == 0:
                index = 4 + m
            else:
                index = sum(list(intervalDict.values())[0:i]) + 4 + m
            MA5Value = sum(df["收盤價"][index-4:index+1])/5
            MA5.append(MA5Value)
        for n in range(intervalDict[SecurityList[i]]-19):
            if i == 0:
                index = 19 + n
            else:
                index = sum(list(intervalDict.values())[0:i]) + 19 + n
            MA20Value = sum(df["收盤價"][index-19:index+1])/20
            MA20.append(MA20Value)
        for o in range(intervalDict[SecurityList[i]]-59):
            if i == 0:
                index = 59 + o
            else:
                index = sum(list(intervalDict.values())[0:i]) + 59 + o
            MA60Value = sum(df["收盤價"][index-59:index+1])/60
            MA60.append(MA60Value)
        for p in range(5):
            MA6.append("NA")
            MA6_BIAS.append("NA")
        for q in range(11):
            MA12.append("NA")
            MA12_BIAS.append("NA")
        for r in range(23):
            MA24.append("NA")
            MA24_BIAS.append("NA")
        for s in range(71):
            MA72.append("NA")
            MA72_BIAS.append("NA")
        for t in range(intervalDict[SecurityList[i]]-5):
            if i == 0:
                index = 5 + t
            else:
                index = sum(list(intervalDict.values())[0:i]) + 5 + t
            MA6Value = sum(df["收盤價"][index-5:index+1])/6
            MA6_BIASValue = round((df["收盤價"][index] - MA6Value)/MA6Value, 4)
            MA6.append(MA6Value)
            MA6_BIAS.append(MA6_BIASValue)
        for u in range(intervalDict[SecurityList[i]]-11):
            if i == 0:
                index = 11 + u
            else:
                index = sum(list(intervalDict.values())[0:i]) + 11 + u
            MA12Value = sum(df["收盤價"][index-11:index+1])/12
            MA12_BIASValue = round((df["收盤價"][index] - MA12Value)/MA12Value, 4)
            MA12.append(MA12Value)
            MA12_BIAS.append(MA12_BIASValue)
        for v in range(intervalDict[SecurityList[i]]-23):
            if i == 0:
                index = 23 + v
            else:
                index = sum(list(intervalDict.values())[0:i]) + 23 + v
            MA24Value = sum(df["收盤價"][index-23:index+1])/24
            MA24_BIASValue = round((df["收盤價"][index] - MA24Value)/MA24Value, 4)
            MA24.append(MA24Value)
            MA24_BIAS.append(MA24_BIASValue)
        for w in range(intervalDict[SecurityList[i]]-71):
            if i == 0:
                index = 71 + w
            else:
                index = sum(list(intervalDict.values())[0:i]) + 71 + w
            MA72Value = sum(df["收盤價"][index-71:index+1])/72
            MA72_BIASValue = round((df["收盤價"][index] - MA72Value)/MA72Value, 4)
            MA72.append(MA72Value)
            MA72_BIAS.append(MA72_BIASValue)
    df["MA6"] = MA6
    df["MA12"] = MA12
    df["MA24"] = MA24
    df["MA72"] = MA72
    df["MA6BIAS"] = MA6_BIAS
    df["MA12BIAS"] = MA12_BIAS
    df["MA24BIAS"] = MA24_BIAS
    df["MA72BIAS"] = MA72_BIAS

    MA5_DIFFE = []
    MA20_DIFFE = []
    MA60_DIFFE = []
    for i in range(len(SecurityList)):
        for j in range(5):
            MA5_DIFFE.append("NA")
        for k in range(20):
            MA20_DIFFE.append("NA")
        for l in range(60):
            MA60_DIFFE.append("NA")
        for m in range(intervalDict[SecurityList[i]]-5):
            if i == 0:
                index = 5 + m
            else:
                index = sum(list(intervalDict.values())[0:i]) + 5 + m
            MA5_DIFFEValue = df["收盤價"][index] - df["收盤價"][index-5]
            MA5_DIFFE.append(MA5_DIFFEValue)
        for n in range(intervalDict[SecurityList[i]]-20):
            if i == 0:
                index = 20 + n
            else:
                index = sum(list(intervalDict.values())[0:i]) + 20 + n
            MA20_DIFFEValue = df["收盤價"][index] - df["收盤價"][index-20]
            MA20_DIFFE.append(MA20_DIFFEValue)
        for o in range(intervalDict[SecurityList[i]]-60):
            if i == 0:
                index = 60 + o
            else:
                index = sum(list(intervalDict.values())[0:i]) + 60 + o
            MA60_DIFFEValue = df["收盤價"][index] - df["收盤價"][index-60]
            MA60_DIFFE.append(MA60_DIFFEValue)
    df["MA5DIFFE"] = MA5_DIFFE
    df["MA20DIFFE"] = MA20_DIFFE
    df["MA60DIFFE"] = MA60_DIFFE
    #------------------------------

    #--------------------DMI指標(動向指標)
    TR = []
    for i in range(len(SecurityList)):
        TR.append("NA")
        for k in range(intervalDict[SecurityList[i]]-1):
            if i == 0:
                index = 1 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 1 + k
            TRvalue = max(df["最高價"][index],df["收盤價"][index-1]) - min(df["最低價"][index],df["收盤價"][index-1])
            TR.append(TRvalue)
    df["TR"] = TR

    TR14 = []
    for i in range(len(SecurityList)):
        for j in range(14):
            TR14.append("NA")
        for k in range(intervalDict[SecurityList[i]]-14):
            if i == 0:
                index = 14 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 14 + k
            if k == 0:
                TR14Value = sum(df["TR"][index-13:index+1])/14
            else:
                TR14Value = TR14[-1] * 13/14 + df["TR"][index] * 1/14
            TR14.append(TR14Value)
    df["TR14"] = TR14

    PositiveDM = []
    NegativeDM = []
    for i in range(len(SecurityList)):
        PositiveDM.append("NA")
        NegativeDM.append("NA")
        for k in range(intervalDict[SecurityList[i]]-1):
            if i == 0:
                index = 1 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 1 + k
            PositiveDMValue = df["最高價"][index] - df["最高價"][index-1]
            NegativeDMValue = df["最低價"][index-1] - df["最低價"][index]
            if (PositiveDMValue > NegativeDMValue) and (PositiveDMValue > 0):
                PositiveDMValue = PositiveDMValue
            else:
                PositiveDMValue = 0
            if (PositiveDMValue < NegativeDMValue) and (NegativeDMValue > 0):
                NegativeDMValue = NegativeDMValue
            else:
                NegativeDMValue = 0
            PositiveDM.append(PositiveDMValue)
            NegativeDM.append(NegativeDMValue)
    df["PositiveDM"] = PositiveDM
    df["NegativeDM"] = NegativeDM

    PositiveDM14 = []
    NegativeDM14 = []
    for i in range(len(SecurityList)):
        for j in range(14):
            PositiveDM14.append("NA")
            NegativeDM14.append("NA")
        for k in range(intervalDict[SecurityList[i]]-14):
            if i == 0:
                index = 14 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 14 + k
            if k == 0:
                PositiveDM14Value = sum(df["PositiveDM"][index-13:index+1])/14
                NegativeDM14Value = sum(df["NegativeDM"][index-13:index+1])/14
            else:
                PositiveDM14Value = PositiveDM14[-1] * 13/14 + df["PositiveDM"][index] * 1/14
                NegativeDM14Value = NegativeDM14[-1] * 13/14 + df["NegativeDM"][index] * 1/14
            PositiveDM14.append(PositiveDM14Value)
            NegativeDM14.append(NegativeDM14Value)
    df["PositiveDM14"] = PositiveDM14
    df["NegativeDM14"] = NegativeDM14

    PositiveDI14 = []
    NegativeDI14 = []
    for i in range(len(SecurityList)):
        for j in range(14):
            PositiveDI14.append("NA")
            NegativeDI14.append("NA")
        for k in range(intervalDict[SecurityList[i]]-14):
            if i == 0:
                index = 14 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 14 + k
            PositiveDI14Value = (df["PositiveDM14"][index]/df["TR14"][index]) * 100
            NegativeDI14Value = (df["NegativeDM14"][index]/df["TR14"][index]) * 100
            PositiveDI14.append(PositiveDI14Value)
            NegativeDI14.append(NegativeDI14Value)
    df["PositiveDI14"] = PositiveDI14
    df["NegativeDI14"] = NegativeDI14

    DX = []
    for i in range(len(SecurityList)):
        for j in range(14):
            DX.append("NA")
        for k in range(intervalDict[SecurityList[i]]-14):
            if i == 0:
                index = 14 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 14 + k
            DXValue = ((abs(df["PositiveDI14"][index] - df["NegativeDI14"][index]))/(df["PositiveDI14"][index] + df["NegativeDI14"][index]))*100
            DX.append(DXValue)
    df["DX"] = DX

    ADX = []
    for i in range(len(SecurityList)):
        for j in range(27):
            ADX.append("NA")
        for k in range(intervalDict[SecurityList[i]]-27):
            if i == 0:
                index = 27 + k
            else:
                index = sum(list(intervalDict.values())[0:i]) + 27 + k
            if k == 0:
                ADXValue = sum(df["DX"][index-13:index+1])/14
            else:
                ADXValue = ADX[-1] * 13/14 + df["DX"][index] * 1/14
            ADX.append(ADXValue)
    df["ADX"] = ADX
    #------------------------------
    return df.to_csv('價量資料庫.csv', encoding="utf-8", index = 0)